In [1]:
# !pip install magnum

In [2]:
# PB_REL="https://github.com/protocolbuffers/protobuf/releases"
# !curl -LO $PB_REL/download/v3.15.8/protoc-3.15.8-linux-x86_64.zip
# !unzip protoc-3.15.8-linux-x86_64.zip -d $HOME/.local
# !export PATH="$PATH:$HOME/.local/bin"

In [ ]:
import json

: 

In [3]:
# !conda install protobuf

In [4]:
from typing import Dict, List, Any

import argparse
from collections import defaultdict
import os
from pathlib import Path
import random
import sys

import numpy as np
import numba
import torch
import tqdm

cwd = os.getcwd()
pwd = os.path.dirname(cwd)
ppwd = os.path.dirname(pwd)

for dir in [cwd, pwd, ppwd]:
    sys.path.insert(1, dir)

from habitat.core.registry import registry
from habitat_baselines.common.environments import get_env_class
from habitat_baselines.config.default import get_config
from habitat_baselines.utils.common import batch_obs
from habitat_baselines.utils.env_utils import construct_envs

from cos_eor.policy.rank import RankModule
from cos_eor.policy.nav import NavModule
from cos_eor.policy.oracle_rank import OracleRankModule
from cos_eor.policy.explore import ExploreModule
from cos_eor.policy.hie_policy import HiePolicy
from cos_eor.env.env import CosRearrangementRLEnv
from cos_eor.task.sensors import *
from cos_eor.task.measures import *

dir_path = "/coc/testnvme/gchhablani3/housekeep/"
os.chdir(dir_path)
output_directory = "logs/baseline_1"
output_path = os.path.join(dir_path, output_directory)
if not os.path.exists(output_path):
    os.mkdir(output_path)

# TODO: Change for all!
config_yaml = './logs/baseline_1/configs/ihlen_1_int.yaml'
tag='ihlen_1_int'

config = get_config(config_yaml)
print(config.TASK_CONFIG.SEED, config.NUM_PROCESSES)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
random.seed(config.TASK_CONFIG.SEED * config.NUM_PROCESSES)
np.random.seed(config.TASK_CONFIG.SEED * config.NUM_PROCESSES)
torch.manual_seed(config.TASK_CONFIG.SEED * config.NUM_PROCESSES)

out_dir = output_directory

config.defrost()

navmesh_file = Path(out_dir)/config.TASK_CONFIG.SIMULATOR.NAVMESH
navmesh_file.parent.mkdir(parents=True, exist_ok=True)
config.TASK_CONFIG.SIMULATOR.NAVMESH = str(navmesh_file)

config.TASK_CONFIG.DATASET.CHECKPOINT_FILE = None
config.freeze()

/srv/flash1/gchhablani3/miniconda3/envs/ovr/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100 1


/coc/testnvme/gchhablani3/housekeep/habitat-lab/habitat_baselines/config/default.py:226: UserWarning: NUM_PROCESSES is depricated and will be removed in a future version.  Use NUM_ENVIRONMENTS instead.  Overwriting NUM_ENVIRONMENTS with NUM_PROCESSES for backwards compatibility.
  "NUM_PROCESSES is depricated and will be removed in a future version."


In [5]:
config.RL.POLICY

Config({'name': 'HiePolicy', 'OBS_TRANSFORMS': Config({'ENABLED_TRANSFORMS': (), 'CENTER_CROPPER': Config({'HEIGHT': 256, 'WIDTH': 256}), 'RESIZE_SHORTEST_EDGE': Config({'SIZE': 256}), 'CUBE2EQ': Config({'HEIGHT': 256, 'WIDTH': 512, 'SENSOR_UUIDS': []}), 'CUBE2FISH': Config({'HEIGHT': 256, 'WIDTH': 256, 'FOV': 180, 'PARAMS': (0.2, 0.2, 0.2), 'SENSOR_UUIDS': []}), 'EQ2CUBE': Config({'HEIGHT': 256, 'WIDTH': 256, 'SENSOR_UUIDS': []})}), 'score_threshold': 0.065, 'oracle': False, 'rearrange_order': 'discovery', 'nav': Config({'name': 'OracleShortestPath'}), 'explore': Config({'name': 'frontier', 'type': 'oracle', 'max_steps_since_new_area': 10000, 'max_steps': 128, 'highres_occ_map_size': 128}), 'rank': Config({'name': 'Oracle', 'file': 'cos_eor/scripts/orm/clip_scores.npy', 'room_select': 'model_scores', 'room_stats_file': 'obj_room_stats.json', 'room_scores_file': 'cos_eor/scripts/orm/clip_obj_room_scores.npy'})})

In [ ]:
envs = construct_envs(
    config, get_env_class(config.ENV_NAME)
)
print("Environments Constructed!")

In [ ]:
observations = envs.reset()
print("Observations Keys: ", observations[0].keys())

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
def display_sample(rgb_obs, semantic_obs=np.array([]), depth_obs=np.array([])):
    from habitat_sim.utils.common import d3_40_colors_rgb

    rgb_img = Image.fromarray(rgb_obs, mode="RGB")

    arr = [rgb_img]
    titles = ["rgb"]
    if semantic_obs.size != 0:
        semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
        semantic_img.putpalette(d3_40_colors_rgb.flatten())
        semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
        semantic_img = semantic_img.convert("RGBA")
        arr.append(semantic_img)
        titles.append("semantic")

    if depth_obs.size != 0:
        depth_img = Image.fromarray((depth_obs / 10 * 255).astype(np.uint8), mode="L")
        arr.append(depth_img)
        titles.append("depth")

    plt.figure(figsize=(12, 8))
    for i, data in enumerate(arr):
        ax = plt.subplot(1, 3, i + 1)
        ax.axis("off")
        ax.set_title(titles[i])
        plt.imshow(data)
    plt.show(block=False)

In [ ]:
observations[0]['depth'].shape

In [ ]:
observations[0]['rgb'].shape

In [ ]:
np.unique(observations[0]['semantic'])

In [ ]:
display_sample(observations[0]['rgb'], observations[0]['semantic'], observations[0]['depth'].squeeze())

In [ ]:
print("# Visible Objects: ", observations[0]['num_visible_objs'])
print("# Visible Receptables: ", observations[0]['num_visible_recs'])

In [ ]:
print(observations[0]['cos_eor'].keys())

In [ ]:
print(observations[0]['cos_eor']['agent_pos'])

In [ ]:
print(len(observations[0]['cos_eor']['recs_pos']))

In [ ]:
print(len(observations[0]['cos_eor']['objs_pos']))

In [ ]:
print(observations[0]['cos_eor']['instance_id_count'])

In [ ]:
print(observations[0]['cos_eor']['sid_class_map'])

In [ ]:
print(observations[0]['cos_eor']['current_mapping'])

In [ ]:
print(observations[0]['cos_eor']['correct_mapping'])